In [1]:
import os

In [2]:
os.chdir('..')

<img src="flow_0.png">

In [3]:
import numpy as np
from flows.flows import Flows

Welcome to the Data Science Package. First create an object as follows:
For example, use the code below to import the flow 0:
flow = Flows(0)
You can define the `categorical_threshold` which is the maximum number of categories that a categorical feature should have before considering it as continuous numeric feature. The default value is 50
For example, use the code below to import the flow 0 with defining the categorical_threshold as 50
flow = Flows(flow_id=0, categorical_threshold=50)


In [4]:
flow = Flows(flow_id=0, categorical_threshold=10)

Please use the following function to read the data
dataframe_dict, columns_set = flow.load_data(path : str, files_list : list)
For example: path = ./data
If your data is in a nested directory, it is better to os.path.join. For example:
path = os.path.join('data', 'flow_0')
For example: files_list = ['train.csv','test.csv']
The output is a dictionary that contains dataframes e.g.

dataframe_dict = {'train': train_dataframe,'test': test_dataframe}
If you want to explore the data you can run one of the following functions: 
1 . flow.exploring_data(dataframe_dict: dict, key_i: str)
For example: flow.exploring_data(dataframe_dict, 'train')
2 . flow.comparing_statistics(dataframe_dict: dict)
For example: flow.comparing_statistics(dataframe_dict)




In [5]:
path = os.path.join('data','flow_0')
files_list = ["train.csv","test.csv"]

In [6]:
dataframe_dict, columns_set = flow.load_data(path, files_list)

A summary of the data sets


,train,test
column type,,
categorical_integer,12,12
categorical_string,43,43
continuous,26,25
date,0,0
json,0,0
other,0,0
total amount,81,80


NOTE: numeric categorical columns that contains more than 10 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
The possible ids are {'Id'}
The possible possible_target are ['SalePrice']
The type of the problem that should be solved {'SalePrice': 'regression'}
If you have categorical features with string labels, Encode the categorical features by applying the following function:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict: dict)


In [7]:
 columns_set

{'train': {'categorical_string': ['MSZoning',
   'Street',
   'Alley',
   'LotShape',
   'LandContour',
   'Utilities',
   'LotConfig',
   'LandSlope',
   'Neighborhood',
   'Condition1',
   'Condition2',
   'BldgType',
   'HouseStyle',
   'RoofStyle',
   'RoofMatl',
   'Exterior1st',
   'Exterior2nd',
   'MasVnrType',
   'ExterQual',
   'ExterCond',
   'Foundation',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Heating',
   'HeatingQC',
   'CentralAir',
   'Electrical',
   'KitchenQual',
   'Functional',
   'FireplaceQu',
   'GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PavedDrive',
   'PoolQC',
   'Fence',
   'MiscFeature',
   'SaleType',
   'SaleCondition'],
  'categorical_integer': ['OverallQual',
   'OverallCond',
   'BsmtFullBath',
   'BsmtHalfBath',
   'FullBath',
   'HalfBath',
   'BedroomAbvGr',
   'KitchenAbvGr',
   'Fireplaces',
   'GarageCars',
   'PoolArea',
   'YrSold'],
  'continuous': ['Id',
   'MSSub

In [8]:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict)

The reference dataframe is: train
******************************
A summary of the data sets


,train,test
column type,,
categorical_integer,52,52
categorical_string,0,0
continuous,29,28
date,0,0
json,0,0
other,0,0
total amount,81,80


NOTE: numeric categorical columns that contains more than 10 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
If you have numeric features, it is a good idea to normalize numeric features.Use the following function for feature normalization:
 dataframe_dict, columns_set = flow.scale_data (dataframe_dict: dict, ignore_columns: list)
For example: ignore_columns = ['id', 'target']


In [9]:
ignore_columns = ['Id', 'SalePrice']

In [10]:
dataframe_dict, columns_set = flow.scale_data(dataframe_dict, ignore_columns)

A summary of the data sets


,train,test
column type,,
categorical_integer,52,52
categorical_string,0,0
continuous,29,28
date,0,0
json,0,0
other,0,0
total amount,81,80


NOTE: numeric categorical columns that contains more than 10 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
If you want to explore the data you can run one of the following functions: 
1 . flow.exploring_data(dataframe_dict: dict, key_i: str)
For example: flow.exploring_data(dataframe_dict, 'train')
2 . flow.comparing_statistics(dataframe_dict: dict)
For example: flow.comparing_statistics(dataframe_dict)


Your features are ready to train the model: 
You can start training the model by applying the following function: 
model_index_list, save_models_dir, y_test = flow.training(parameters)
parameters = {
     "data": {
         "train": {"features": train_dataframe, "target": train_target}, 
         "valid": {"features": valid_dataframe, "target": valid_target}, 
         "test": {"features": test_dataframe, "target": test_target}, 
     }, 
   

In [11]:
flow.exploring_data(dataframe_dict, "train")

interactive(children=(IntSlider(value=0, description='column_i', max=80), Output()), _dom_classes=('widget-int…

In [12]:
flow.comparing_statistics(dataframe_dict)

interactive(children=(IntSlider(value=0, description='feature_nr', max=80), Output()), _dom_classes=('widget-i…

In [13]:
ignore_columns = ["Id", "SalePrice"]
columns = dataframe_dict["train"].columns
train_dataframe = dataframe_dict["train"][[x for x in columns_set["train"]["continuous"] if x not in ignore_columns]]
test_dataframe = dataframe_dict["test"][[x for x in columns_set["train"]["continuous"] if x not in ignore_columns]]
train_target = dataframe_dict["train"]["SalePrice"]

In [14]:
parameters = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # fold_nr:5 , "split_ratios": 0.3 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "Ridge linear regression",
              "hyperparameters": {"alpha": "optimize",  # alpha:optimize
                                  },
              },
    "metrics": ["r2_score"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [15]:
model_index_list, save_models_dir, y_test = flow.training(parameters)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
fold_nr. 1
{'r2_score (train.train)': 73.74, 'r2_score (train.validation)': 78.22}
fold_nr. 2
{'r2_score (train.train)': 78.62, 'r2_score (train.validation)': 56.76}
fold_nr. 3
{'r2_score (train.train)': 74.61, 'r2_score (train.validation)': 76.83}
fold_nr. 4
{'r2_score (train.train)': 76.44, 'r2_score (train.validation)': 70.23}
fold_nr. 5
{'r2_score (train.train)': 74.09, 'r2_score (train.validation)': 79.26}


,fold_1,fold_2,fold_3,fold_4,fold_5,mean
r2_score (train.train),73.74,78.62,74.61,76.44,74.09,75.50
r2_score (train.validation),78.22,56.76,76.83,70.23,79.26,72.26


,model 1,model 2,model 3,model 4,model 5,mean
r2_score (train),74.78,74.41,75.04,75.1,75.01,74.868


This is the end of the flow


In [ ]:
parameters_lighgbm = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # fold_nr:5 , "split_ratios": 0.3 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "lightgbm",
              "hyperparameters": dict(objective='regression', metric='root_mean_squared_error', num_leaves=5,
                                      boost_from_average=True,
                                      learning_rate=0.05, bagging_fraction=0.99, feature_fraction=0.99, max_depth=-1,
                                      num_rounds=10000, min_data_in_leaf=10, boosting='dart')
              },
    "metrics": ["mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list, save_models_dir, y_test = flow.training(parameters_lighgbm)

In [ ]:
parameters_xgboost = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # fold_nr:5 , "split_ratios": 0.3 # "split_ratios":(0.3,0.2)
    },
    "model": {"type": "xgboost",
              "hyperparameters": {'max_depth': 5, 'eta': 1, 'eval_metric': "rmse", "num_round": 100}
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [ ]:
model_index_list, save_models_dir, ytest = flow.training(parameters_xgboost)

In [ ]:
parameters_sklearn = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "split",
        "split_ratios": 0.2,
        "stratify": False  # set to True only for classification tasks
    },
    "model": {
        "type": "sklearn.ensemble.RandomForestRegressor",
        "hyperparameters": {
            'params_grid':{
                'criterion': ["mse", "mae"],
                'max_depth': [5, 10, 15, 999],
                'min_samples_leaf': [4, 1],
                'max_depth': [4, 8, 12],
            },
            'params_fixed': {
                'min_samples_split': 10, 
                'random_state': 11
            },
            'params_cv': {
                'n_splits': 5, 
                'shuffle': True, 
                'random_state': 11
            },
            'objective': 'regression',  # 'classification'
            "grid_search_scoring": ['r2', 'neg_mean_squared_error']
        },
    },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

model_index_list, save_models_dir, y_test = flow.training(parameters_sklearn)

In [ ]:
parameters_sklearn = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "split",
        "split_ratios": 0.2,
        "stratify": False  # set to True only for classification tasks
    },
    "model": {
        "type": "sklearn.linear_model.ElasticNet",
        "hyperparameters": {
            'params_grid':{
                'alpha': np.logspace(-3,3,7),
                'l1_ratio': np.linspace(0, 0., num=4)+0.01
            },
            'params_fixed': {
                'normalize': True, 
                'max_iter': 2000,
                'random_state': 11
            },
            'params_cv': {
                'n_splits': 5, 
                'shuffle': True, 
                'random_state': 11
            },
            'objective': 'regression',  # 'classification'
            "grid_search_scoring": ['r2', 'neg_mean_squared_error']
        },
    },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

model_index_list, save_models_dir, y_test = flow.training(parameters_sklearn)